# 3D Wannier calculations

In [1]:
# Directives on Jupyter
%reset
# Auto reload bootstrap.py
%load_ext autoreload
%autoreload 2

import numpy as np

In [15]:
from wannier import *
import numpy as np
from positify import positify

N = 20
R0 = np.array([3, 3, 7.2])
dvr = Wannier(
    N,
    R0,
    lattice=np.array([3, 3], dtype=int),
    #   lc=(1690, 1520),
    dim=3,
    avg=1 / 2,
    sparse=True,
    symmetry=True)

E, W, parity = eigen_basis(dvr)

A, U = optimization(dvr, E, W, parity)
t_sb = np.array([])
for i in range(dvr.bands):
    mu = np.diag(A[i])  # Diagonals are mu_i
    t = -(A[i] - np.diag(mu))  # Off-diagonals are t_ij
    print(f'mu_i,{i+1} = ', np.real(mu))
    print(f't_ij,{i+1} =', abs(t))
    t_sb = np.append(t_sb, abs(t[0, 1]))
V = interaction(dvr, U, W, parity)
print('U_i =', abs(V))

DVR: dx=[0.15 0.15 0.36]w is set.
DVR: n=[20 20 20] is set.
DVR: R0=[3.  3.  7.2]w is set.
['x' 'y' 'z']-reflection symmetry is used.
param_set: trap parameter V0=104.52kHz w=1000nm
lattice: Full lattice sizes: [3 3 1]
lattice: lattice constants: [1.52 1.69 1.52]w
DVR: dx=[0.15 0.15 0.36]w is set.
DVR: n=[30 31 20] is set.
DVR: R0=[4.52 4.69 7.2 ]w is set.
H_op: n=[30 31 20] dx=[0.15 0.15 0.36]w p=[1 1 1] Gaussian sparse diagonalization is enabled. Lowest 9 states are to be calculated.
H_op: n=[30 31 20] dx=[0.15 0.15 0.36]w p=[1 1 1] Gaussian operator constructed.
H_solver: diagonalize sparse hermitian matrix.
H_solver: Gaussian Hamiltonian solved. Time spent: 1.25s.
H_solver: eigenstates memory usage: 1.43 MiB.
H_op: n=[30 31 20] dx=[0.15 0.15 0.36]w p=[ 1 -1  1] Gaussian sparse diagonalization is enabled. Lowest 9 states are to be calculated.
H_op: n=[30 31 20] dx=[0.15 0.15 0.36]w p=[ 1 -1  1] Gaussian operator constructed.
H_solver: diagonalize sparse hermitian matrix.
H_solver: G

| 1D x chain | t (kHz) | $\mu$ (kHz) | U (kHz) |
| -- | ----- | ----- | ----- |
| 2 | 0.33459339 | -35.15147705 | 1.16669761 |
| 3 | 0.29154572 | -35.15989359 -36.15465807 | 1.16075052 1.08831435 |
| 4 | 0.29111124 0.24774003 | -35.16008983 -36.16069493 | 1.16018491 1.08606769 |
| 5 | 0.291103027 0.247220317 | -35.1600951  -36.16081958 -36.1667645 | 1.15841229 1.08268238 1.08521976 |


| 1D y chain | t (kHz) | $\mu$ (kHz) | U (kHz) |
| -- | ----- | ----- | ----- |
| 2 | 0.09926161 | -34.39925269 | 1.23623438 |
| 3 | 0.09280767 | -34.39959408 -34.83640662 | 1.23609266 1.18905493 |
| 4 | 0.0927989793 0.0864121562 | -34.39959504 -34.83669976 | 1.238437   1.19090978 |
| 5 | 0.0927989561 0.0864030213 | -34.39959504 -34.83670055 -34.83699298 | 1.24381596 1.18095209 1.19415962 |


| 2D Plaquett | t (kHz) | $\mu$ (kHz) | U (kHz) |
| -- | ----- | ----- | ----- |
| 2x2 | 0.33364721 0.09660292 | -35.6348516 | 1.11173354 |
| 3x3 | 0.0904338849 0.290634614 0.289613446 0.0879192624 | -35.6435493 -36.08746291 -36.6445585 -37.09460885 | 1.10585854 1.06025591 1.03430202 0.98913985 |

In [1]:
from wannier import *
import numpy as np
from positify import positify
from scipy.stats import gmean

N = 20
R0 = np.array([3, 3, 7.2])
dvr = Wannier(N,
              R0,
              lattice=np.array([8], dtype=int),
              lc=(1350, 1520),
              trap=(50, 930),
              laser=770,
              dim=3,
              avg=1,
              sparse=True,
              symmetry=True)

E, W, parity = eigen_basis(dvr)

A, U = optimization(dvr, E, W, parity)
t_sb = np.array([])
for i in range(dvr.bands):
    mu = np.diag(A[i])  # Diagonals are mu_i
    t = -(A[i] - np.diag(mu))  # Off-diagonals are t_ij
    print(f'mu_i,{i+1} = ', np.real(mu))
    print(f't_ij,{i+1} =', abs(t))
    t_sb = np.append(t_sb, abs(t[0, 1]))
V = interaction(dvr, U, W, parity)
print('U_i =', abs(V))

DVR: dx=[0.15 0.15 0.36]w is set.
DVR: n=[20 20 20] is set.
DVR: R0=[3.  3.  7.2]w is set.
['x' 'y' 'z']-reflection symmetry is used.
param_set: trap parameter V0=50kHz w=930nm
lattice: Full lattice sizes: [8 1 1]
lattice: lattice constants: [1.4516129 1.6344086 1.4516129]w
DVR: dx=[0.15 0.15 0.36]w is set.
DVR: n=[53 20 20] is set.
DVR: R0=[8.08064516 3.         7.2       ]w is set.
H_op: n=[53 20 20] dx=[0.15 0.15 0.36]w p=[1 1 1] Gaussian sparse diagonalization is enabled. Lowest 8 states are to be calculated.
H_op: n=[53 20 20] dx=[0.15 0.15 0.36]w p=[1 1 1] Gaussian operator constructed.
H_solver: diagonalize sparse hermitian matrix.
H_solver: Gaussian Hamiltonian solved. Time spent: 2.24s.
H_solver: eigenstates memory usage: 1.45 MiB.
H_op: n=[53 20 20] dx=[0.15 0.15 0.36]w p=[-1  1  1] Gaussian sparse diagonalization is enabled. Lowest 8 states are to be calculated.
H_op: n=[53 20 20] dx=[0.15 0.15 0.36]w p=[-1  1  1] Gaussian operator constructed.
H_solver: diagonalize sparse h

In [2]:
from wannier import *
import numpy as np
from positify import positify
from scipy.stats import gmean

N = 20
R0 = np.array([3, 3, 7.2])
w = np.sqrt(2 / (1 / 930**2 + 1 / 1250**2))
dvr = Wannier(N,
              R0=R0,
              lattice=np.array([8], dtype=int),
              lc=(1350, 1520),
              trap=(50, w),
              laser=770,
              dim=3,
              avg=1,
              sparse=True,
              symmetry=True)

E, W, parity = eigen_basis(dvr)

A, U = optimization(dvr, E, W, parity)
t_sb = np.array([])
for i in range(dvr.bands):
    mu = np.diag(A[i])  # Diagonals are mu_i
    t = -(A[i] - np.diag(mu))  # Off-diagonals are t_ij
    print(f'mu_i,{i+1} = ', np.real(mu))
    print(f't_ij,{i+1} =', abs(t))
    t_sb = np.append(t_sb, abs(t[0, 1]))
V = interaction(dvr, U, W, parity)
print('U_i =', abs(V))

DVR: dx=[0.15 0.15 0.36]w is set.
DVR: n=[20 20 20] is set.
DVR: R0=[3.  3.  7.2]w is set.
['x' 'y' 'z']-reflection symmetry is used.
param_set: trap parameter V0=50kHz w=1055.2060816556739nm
lattice: Full lattice sizes: [8 1 1]
lattice: lattice constants: [1.27937094 1.44047691 1.27937094]w
DVR: dx=[0.15 0.15 0.36]w is set.
DVR: n=[49 20 20] is set.
DVR: R0=[7.4777983 3.        7.2      ]w is set.
H_op: n=[49 20 20] dx=[0.15 0.15 0.36]w p=[1 1 1] Gaussian sparse diagonalization is enabled. Lowest 8 states are to be calculated.
H_op: n=[49 20 20] dx=[0.15 0.15 0.36]w p=[1 1 1] Gaussian operator constructed.
H_solver: diagonalize sparse hermitian matrix.
H_solver: Gaussian Hamiltonian solved. Time spent: 1.24s.
H_solver: eigenstates memory usage: 1.35 MiB.
H_op: n=[49 20 20] dx=[0.15 0.15 0.36]w p=[-1  1  1] Gaussian sparse diagonalization is enabled. Lowest 8 states are to be calculated.
H_op: n=[49 20 20] dx=[0.15 0.15 0.36]w p=[-1  1  1] Gaussian operator constructed.
H_solver: diago

Use w=930nm is somewhat reasonable for tunneling but not for interaction U. But using this the interaction is closer to exp't value compared with the other mean.

TODO: find out why w>1000nm cannot converge.